In [ ]:
pip install requests boto3 pyspark schedule

In [ ]:
import requests
import boto3
import json
import time
import schedule
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


In [ ]:
api_endpoint = 'https://api.mainnet-beta.solana.com'
s3_bucket_name = 'solanatransactions'
aws_access_key_id = 'XXXXXXXXXXXXXXXXXXXX'
aws_secret_access_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
aws_region = 'us-east-2'

In [ ]:
helius_api_endpoint = 'https://mainnet.helius-rpc.com/?api-key=e07245d6-faf6-4d44-9467-a5f9df5b42b1'
helius_params =     {
      "jsonrpc": "2.0",
      "id": "text",
      "method": "getAsset",
      "params": {
        "id":"5SVG3T9CNQsm2kEwzbRq6hASqh1oGfjqTtLXYUibpump"
      }
    }

In [ ]:
wallets = [""]

In [ ]:
parameters =   {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "getSignaturesForAddress",
    "params": [
      "AVAZvHLR2PcWpDf8BXY4rVxNHYRBytycHkcB5z5QNXYm",
      {
        "limit": 10
      }
    ]
  }

In [ ]:
params2 = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "getTransaction",
    "params": [
      "4smXaYpz1fWGq62DLs8P8sRk51MeraBH13qFcRJLW7oDx87U5FcTqoUSRdU5XwdZhDUrrZC2EB77ja7XowpEpvGg",
      "json"
    ]
  }

In [ ]:
signatures = {'' : []
              }

def get_signatures_data(wallet):
    parameters['params'][0] = wallet
    response = requests.post(api_endpoint,headers = {'Content-Type' : 'application/json'}, json=parameters)
    data = json.loads(response.text)
    for i in data['result']:
        signatures[wallet].append(i['signature'])
    return signatures


In [ ]:
def getTransactions(signatures, wallet):
    for s in signatures[wallet]:
      params2['params'][0] = s
      response = requests.post(api_endpoint, headers = {'Content-Type' : 'application/json'},  json=params2)
      time.sleep(10)
      res = getTransactionsResults(response.text, wallet)
      print(res) if res != None else None
      fin.append(res) if res != None else None



In [ ]:
def post_data_to_s3(s3_bucket_name, file_name):
    # Initialize S3 client
    object_name = file_name

    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=aws_region
    )

    s3_client.upload_file(file_name, s3_bucket_name, object_name)

def create_file(filename, data):
    with open(filename, 'w') as f:
        f.write(data)

In [ ]:
def getDataFromMint(mint):
    helius_params['params']['id'] = mint
    response = requests.post(helius_api_endpoint, headers = {'Content-Type' : 'application/json'},  json=helius_params)
    data = json.loads(response.text)
    return data['result']

In [ ]:
def getTransactionsResults(transaction, wallet):
    result = []
    data = json.loads(transaction)
    if 'result' in data:
        if 'meta' in data['result']:
            if 'postTokenBalances' in data['result']['meta'] and len(data['result']['meta']['postTokenBalances']) > 0:
              mintData = getDataFromMint(data['result']['meta']['postTokenBalances'][0]['mint'])
              for i in data['result']['meta']['postTokenBalances']:
                   if 'token_info' in mintData:
                    if 'price_info' in mintData['token_info']:
                      if 'price_per_token' in mintData['token_info']['price_info']:
                          result.append({
                              'datetime': datetime.utcfromtimestamp(data['result']['blockTime']).strftime('%Y-%m-%d %H:%M:%S'),
                              'owner': i['owner'],
                              'name': mintData['content']['metadata']['name'],
                              'amount': i['uiTokenAmount']['uiAmount'],
                              'USD': mintData['token_info']['price_info']['price_per_token'],
                              'amount_in_USD': (i['uiTokenAmount']['uiAmount']) * mintData['token_info']['price_info']['price_per_token']
                          })
    return result if len(result) > 0 else None


In [ ]:
def read_files_from_s3(bucket_name):
  s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=aws_region)
  objects = s3.list_objects(Bucket=bucket_name)
  for obj in objects['Contents']:
    key = obj['Key']
    response = s3.get_object(Bucket=bucket_name, Key=key)
    file_content = response['Body'].read()
    print(file_content)
    return file_content

In [ ]:
def json_to_df(json_data):
    spark = SparkSession.builder.appName("JSON to DataFrame").getOrCreate()
    schema = StructType([
    StructField('datatime1', StringType(), True),
    StructField('owner1', StringType(), True),
    StructField('name1', StringType(), True),
    StructField('amount1', IntegerType(), True),
    StructField('USD1', IntegerType(), True),
    StructField('amount_in_USD1', IntegerType(), True),
    StructField('datatime2', StringType(), True),
    StructField('owner2', StringType(), True),
    StructField('name2', StringType(), True),
    StructField('amount2', IntegerType(), True),
    StructField('USD2', IntegerType(), True),
    StructField('amount_in_USD2', IntegerType(), True),
    ])
    df = spark.createDataFrame(json_data).schema(schema)
    df.filter(df['amount_in_USD'] > 150)
    df.show()
    return df

In [ ]:
fin = []
def job():
  for w in wallets:
      get_signatures_data(w)
      time.sleep(10)
      getTransactions(signatures, w)
  tim = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
  create_file(tim,json.dumps(fin))
  post_data_to_s3(s3_bucket_name, tim)
  signatures.clear()
  fin.clear()

schedule.every().hour.do(job)

def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(1)

run_scheduler()

def job2():
  dataset = read_files_from_s3(s3_bucket_name)
  json_to_df(dataset)

schedule.every().hour.do(job2)

def run_scheduler1():
    while True:
        schedule.run_pending()
        time.sleep(1)

run_scheduler1()



[{'datetime': '2024-08-05 22:22:22', 'owner': '9J9Dw4gZgDcSRJxtU1gvkFRt3bC8BTKSFCjzEFesQvLv', 'name': 'archie', 'amount': 2516512.628128, 'USD': 2.51551e-05, 'amount_in_USD': 63.30312681182265}, {'datetime': '2024-08-05 22:22:22', 'owner': 'AVAZvHLR2PcWpDf8BXY4rVxNHYRBytycHkcB5z5QNXYm', 'name': 'archie', 'amount': 1.0, 'USD': 2.51551e-05, 'amount_in_USD': 2.51551e-05}]
[{'datetime': '2024-08-05 21:55:15', 'owner': 'AVAZvHLR2PcWpDf8BXY4rVxNHYRBytycHkcB5z5QNXYm', 'name': 'Pheonix', 'amount': 1000000.0, 'USD': 3.84638e-06, 'amount_in_USD': 3.8463799999999995}, {'datetime': '2024-08-05 21:55:15', 'owner': 'Hut54FGgWG3zLoTvAo4iiMW3AWNLhLcRHwPe6hGxtVEb', 'name': 'Pheonix', 'amount': 30252427.184466, 'USD': 3.84638e-06, 'amount_in_USD': 116.36233087378632}]
[{'datetime': '2024-08-05 21:27:03', 'owner': 'AVAZvHLR2PcWpDf8BXY4rVxNHYRBytycHkcB5z5QNXYm', 'name': 'avoter: teh lost airboner', 'amount': 150784.8497949, 'USD': 1.1549962e-06, 'amount_in_USD': 0.17415592853068027}, {'datetime': '2024-08